In [28]:
import torch 
from torch.nn import Embedding
from torch.nn import Linear
import numpy as np

In [29]:
torch.manual_seed(1)

### 1.Embedding

#### 1.1 Embedding的作用

Embedding层的作用是将有限集合中的元素，转变成指定size的向量。这个有限集合可以使NLP中的词汇表，可以使分类任务中的label，当然无论是什么，最终都要以元素索引传递给Embedding。例如，将包含3个元素的词汇表W={'优', '良', '差'}中的每个元素转换为5维向量。如下所示：

In [30]:
# 先定义一个Embedding层：
emb = Embedding(num_embeddings=3, embedding_dim=5)

In [31]:
emb

Embedding(3, 5)

In [32]:
# 转换第一个元素
emb(torch.tensor([0],dtype=torch.int64))

tensor([[ 0.6614,  0.2669,  0.0617,  0.6213, -0.4519]],
       grad_fn=<EmbeddingBackward0>)

In [33]:
# 转换第二个元素
emb(torch.tensor([1],dtype=torch.int64))

tensor([[-0.1661, -1.5228,  0.3817, -1.0276, -0.5631]],
       grad_fn=<EmbeddingBackward0>)

In [34]:
# 转换第三个元素
emb(torch.tensor([2],dtype=torch.int64))

tensor([[-0.8923, -0.0583, -0.1955, -0.9656,  0.4224]],
       grad_fn=<EmbeddingBackward0>)

In [35]:
# 如果超出词库规模，就会产生异常错误：
# 转换第四个元素
emb(torch.tensor([3],dtype=torch.int64))

IndexError: index out of range in self

初始时，所有向量表示都是随机的，但却并非一成不变的，例如在NLP任务中，随着网络的训练，表示'优'与'良'的两个向量相似度会逐渐减小，而表示'优'与'差'的两个向量相似度会逐渐增大。

#### 1.2 Embedding的用法

接下来我们详细说说pytorch中Embedding层的使用方法。Embedding类主要参数如下：

num_embeddings (int) - 嵌入字典的大小，即共有多少个元素需要转换

embedding_dim (int) - 每个嵌入向量的大小，即转换后获得向量的size

padding_idx (int, optional) - 如果提供的话，输出遇到此下标时用零填充

max_norm (float, optional) - 如果提供的话，会重新归一化词嵌入，使它们的范数小于提供的值

norm_type (float, optional) - 对于max_norm选项计算p范数时的p

scale_grad_by_freq (boolean, optional) - 如果提供的话，会根据字典中单词频率缩放梯度

weight weight (Tensor) -形状为(num_embeddings, embedding_dim)的模块中可学习的权值

 
Embedding是怎么实现的呢？其实，在初始化Embedding层时，Embedding会根据默认随机初始化num_embeddings * embedding_dim的正态分布的权重。以上面例子为例，我们看看它的参数：

In [36]:
# 先定义一个Embedding层：
emb = Embedding(num_embeddings=10, embedding_dim=5)
emb.weight

Parameter containing:
tensor([[ 1.8793, -0.0721,  0.1578, -0.7735,  0.1991],
        [ 0.0457,  0.1530, -0.4757, -0.1110,  0.2927],
        [-0.1578, -0.0288,  2.3571, -1.0373,  1.5748],
        [-0.6298, -0.9274,  0.5451,  0.0663, -0.4370],
        [ 0.7626,  0.4415,  1.1651,  2.0154,  0.1374],
        [ 0.9386, -0.1860, -0.6446,  1.5392, -0.8696],
        [-3.3312, -0.7479, -0.0255, -1.0233, -0.6540],
        [ 0.7317, -1.4344, -0.5008,  0.1716, -0.1600],
        [-0.5047, -1.4746, -1.0412,  0.7323, -1.0483],
        [-0.4709,  0.2911,  1.9907, -0.9247, -0.9301]], requires_grad=True)

仔细观察这些权重值，每一行都与上方{'优', '良', '差'}对应。当我们在emb中输入张量torch.tensor([0])时，输出了第一行，当我们在emb中输入张量torch.tensor([1])时，输出了第二行。所以，我们可以猜测，Embedding的工作原理就是初始化一个指定shape的矩阵，在进行转换是，根据输入的tensor值，索引矩阵的行。确实如此，Embedding源码就是这么做的。

当然，Embedding的权重参数也不一定非得随机初始化，也可以手动指定。如下所示，我们先手动初始化一个3 * 5的矩阵，然后将其作为Embedding的权重参数：

In [37]:
# 随机初始化一个3 * 5 的矩阵
emb_weight = torch.rand(3, 5, requires_grad=True)

# 这里需要注意，手动初始化参数时，最好设置requires_grad=True，后续训练时才能更新权重。
emb_weight

tensor([[0.2873, 0.3486, 0.9579, 0.4075, 0.7819],
        [0.7165, 0.1768, 0.0748, 0.9799, 0.5261],
        [0.8427, 0.6036, 0.6608, 0.8735, 0.9741]], requires_grad=True)

In [38]:
# 通过这个预先定义的矩阵，初始化Embedding层
emb2 = Embedding.from_pretrained(emb_weight)

In [39]:
emb2.weight

Parameter containing:
tensor([[0.2873, 0.3486, 0.9579, 0.4075, 0.7819],
        [0.7165, 0.1768, 0.0748, 0.9799, 0.5261],
        [0.8427, 0.6036, 0.6608, 0.8735, 0.9741]])

In [40]:
# 转换第一个元素
emb2(torch.tensor([0],dtype=torch.int64))

tensor([[0.2873, 0.3486, 0.9579, 0.4075, 0.7819]])

In [41]:
# 查看所有权重参数
emb2.weight

Parameter containing:
tensor([[0.2873, 0.3486, 0.9579, 0.4075, 0.7819],
        [0.7165, 0.1768, 0.0748, 0.9799, 0.5261],
        [0.8427, 0.6036, 0.6608, 0.8735, 0.9741]])

这种手动指定参数参数话Embedding层的方式在迁移学习中非常实用，例如在NLP任务中，我们可以使用开源的词向量模型进行初始化，使得我们的模型更快收敛。

### 2 Linear层

#### 2.1 Linear层的作用
Linear层是最古老、最基础的一种网络结构了，作用是对输入向量进行线性变换，输出指定size的向量。例如，将size为3的向量，转为size为5的向量：

In [42]:
# 初始化一个Linear层
lin = Linear(in_features=3, out_features=5)

In [43]:
# 随机初始化一个size为3的向量
x = torch.rand(3)
x

tensor([0.8313, 0.8116, 0.8553])

In [44]:
x.shape

torch.Size([3])

In [45]:
# 经Linear层进行转换
y = lin(x)
y

tensor([ 0.4413,  0.8174, -0.9255,  0.5708, -0.3130], grad_fn=<ViewBackward0>)

In [46]:
y.shape

torch.Size([5])

#### 2.2 Linear的用法
 
Linear类就3个参数：

in_features：指的是输入张量的size
out_features：指的是输出张量的size
bias：是否使用偏置，默认为True，表示使用

In [47]:
w = lin.weight
w

Parameter containing:
tensor([[-0.3831,  0.0722,  0.4309],
        [ 0.4183,  0.3587, -0.4178],
        [-0.4158, -0.3492,  0.0725],
        [ 0.5754, -0.3647,  0.3077],
        [-0.3196, -0.5428, -0.1227]], requires_grad=True)

In [48]:
b = lin.bias
b

Parameter containing:
tensor([ 0.3327,  0.5360, -0.3586,  0.1253,  0.4982], requires_grad=True)

In [49]:
x.matmul(w.T) + b

tensor([ 0.4413,  0.8174, -0.9255,  0.5708, -0.3130], grad_fn=<AddBackward0>)

In [50]:
# Linear层的参数也可以进行手动修改，不过必须转为Parameter

lin_weight = torch.rand(3, 5, requires_grad=True)
lin_weight

from torch.nn import Parameter
lin.weight = Parameter(lin_weight)

### 3.Embedding与Linear的区别

- Embedding只针对数据集规模有限的离散型数据，Linear即可用于离散型数据，也可用于连续型数据，且对数据集规模无限制。对于Embedding能实现的功能，Liner都能实现，只不过需要先进性一次手动one-hot编码。

- Embedding本质是通过元素的索引，获取矩阵对应行作为输出，而Linear本质是矩阵相乘。